In [ ]:
#### Coordinate MLP (Tutte cutting)
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "10000",
             "--val_interval", "20", "--data_file", "cylinder.json", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--initjinput", "--noencoder",
              "--lossdistortion", "dirichlet", "--projectname", "coarsetutte", "--lr", "1e-5",
              "--targets_per_batch", "16", "--workers", "1", "--outputdir", "./outputs/learning/coarse_tutte",
              "--accumulate_grad_batches", "1", "--opttrans", "--min_cuts", "1", "--max_cuts", "6",
              "--lossgradientstitching", "l2",
              "--top_k_eig", "40"]

basestr = "python ./training_scripts/train.py " + ' '.join(basestr)
inits = ['tutte']
datadirs = ['coarsecylinder', 'coarsecylinder_nocut']
lrs = ['1e-5']

f = open("./slurm/cylinder_coarse_tutte", 'w')
for data in datadirs: 
    for init in inits:
        expname = f"coordmlp_{data}_init{init}"
        expstr = basestr + f" --init {init} --ninit -1 --root_dir_train ./data/{data} --root_dir_test ./data/{data} --expname {expname}"
        f.write(expstr + "\n")
f.close() 

In [1]:
#### Dense MLP
## data type 
## stitching type 
## global rotation sampling/random/basis sampling 
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "10000",
             "--val_interval", "20", "--data_file", "cylinder.json", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--initjinput", "--noencoder",
              "--lossdistortion", "dirichlet", "--projectname", "dense", "--lr", "1e-5",
              "--targets_per_batch", "16", "--workers", "1", "--outputdir", "./outputs/dense_v2",
              "--accumulate_grad_batches", "1", "--opttrans",
              "--dense", "input", "--lossgradientstitching", "l2",
              "--top_k_eig", "40"]

basestr = "python ./training_scripts/train.py " + ' '.join(basestr)
inits = ['isometric']
basistypes = ['rot', 'basis', 'global']
gradclips = [True, False]
datadirs = ['coarsecylinder', 'coarsecylinder_nocut']
lrs = ['1e-5']

f = open("./slurm/cylinder_dense", 'w')
for data in datadirs: 
    for init in inits:
        for basis in basistypes: 
            expname = f"{data}_init{init}_basis{basis}"
            expstr = basestr + f" --init {init} --ninit -1 --root_dir_train ./data/{data} --root_dir_test ./data/{data} --expname {expname}"
            expstr += f" --basistype {basis}"
            f.write(expstr + "\n")
f.close() 

In [2]:
#### Coordinate MLP
## data type 
## stitching type 
## global rotation sampling/random/basis sampling 
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "10000",
             "--val_interval", "20", "--data_file", "cylinder.json", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--initjinput", "--noencoder",
              "--lossdistortion", "dirichlet", "--projectname", "dense", "--lr", "1e-5",
              "--targets_per_batch", "16", "--workers", "1", "--outputdir", "./outputs/coordmlp",
              "--accumulate_grad_batches", "1", "--opttrans",
              "--lossgradientstitching", "l2",
              "--top_k_eig", "40"]

basestr = "python ./training_scripts/train.py " + ' '.join(basestr)
inits = ['isometric']
basistypes = ['rot', 'basis', 'global']
gradclips = [True, False]
datadirs = ['coarsecylinder', 'coarsecylinder_nocut']
lrs = ['1e-5']

f = open("./slurm/cylinder_coarse", 'w')
for data in datadirs: 
    for init in inits:
        for basis in basistypes: 
            expname = f"coordmlp_{data}_init{init}_basis{basis}"
            expstr = basestr + f" --init {init} --ninit -1 --root_dir_train ./data/{data} --root_dir_test ./data/{data} --expname {expname}"
            expstr += f" --basistype {basis}"
            f.write(expstr + "\n")
f.close() 

In [7]:
#### Hyperparam search: neural optimization w/ initj input
## - distortion loss: arap/dirichlet 
## - initialization: tutte, isometric 
## - stitchloss schedule: T/F (tutte: 0->1, isometric: 1->2)
## - l0relaxation + schedule: T/F

basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "20000",
             "--val_interval", "20", "--data_file", "cylinder.json", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--lr", "1e-5", "--ninit", "1", "--initjinput",
              "--targets_per_batch", "16", "--workers", "8", "--outputdir", "./outputs/neuralopt",
              "--accumulate_grad_batches", "1", "--opttrans"]
basestr = "python ./training_scripts/train.py " + ' '.join(basestr)
inits = ['isometric', 'tutte']
datadirs = ['cylinder', 'cylinder_nocut']
distortions = ['dirichlet', 'arap']
stitchschedules = [True, False]
l0relaxes = [True, False]

f = open("slurm/cylinder_neuralopt", 'w')
for data in datadirs: 
    for distortion in distortions: 
        for init in inits:
            for stitchschedule in stitchschedules:
                for l0 in l0relaxes:
                    expname = f"{data}_dist{distortion}_init{init}_stitchsched{stitchschedule}_l0{l0}"
                    expstr = basestr + f" --init {init} --lossgradientstitching l2 --root_dir_train ./data/{data} --root_dir_test ./data/{data} --expname {expname}"
                    expstr += f" --lossdistortion {distortion}"
                    if stitchschedule: 
                        if init == "isometric":
                            expstr += f" --stitchloss_schedule linear --stitchlossweight_min 1 --stitchlossweight_max 2"
                        if init == "tutte":
                            expstr += f" --stitchloss_schedule linear --stitchlossweight_min 0 --stitchlossweight_max 1"
                    if l0: 
                        expstr += f" --seploss_schedule --stitchrelax"
                    f.write(expstr + "\n")
f.close() 

In [3]:
# Multiple init learning experiment  
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "10000",
             "--val_interval", "50", "--data_file", "cylinder.json", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--lr", "1e-5", "--workers", "8", "--opttrans",
              "--lossdistortion", "dirichlet", "--targets_per_batch", "16", "--cuteps", "0.5",  
              "--accumulate_grad_batches", "1", "--initjinput", "--outputdir", "outputs/learning"]

basestr = "python ./training_scripts/train.py " + ' '.join(basestr)
inits = ['isometric']
losses = [None, 'l2']
basistypes = ['basis', 'rot']
# ninits = [1,2,5,10,20,100]
ninit_to_ratio = {1:1, 2:1, 5:1, 10: 0.2, 20: 0.1, 100: 0.1}
datadirs = ['cylinder', 'cylinder_nocut']

f = open("slurm/cylinder_n", 'w')
for data in datadirs: 
    for init in inits:
        for loss in losses:
            for basis in basistypes: 
                expname = f"{data}_init{init}_loss{loss}_basis{basis}"
                expstr = basestr + f" --init {init} --ninit -1 --root_dir_train ./data/{data} --root_dir_test ./data/{data} --expname {expname}"
                expstr += f" --basistype {basis}"
                # expstr += f" --valrenderratio {ninit_to_ratio[ninit]}"
                if loss:
                    expstr += f" --lossgradientstitching {loss}"
                else:
                    expstr += f" --lossedgeseparation"
                f.write(expstr + "\n")
f.close() 

In [2]:
# Gradient stitching experiments 
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "3000",
             "--val_interval", "20", "--data_file", "cylinder.json", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--lr", "0.0001",
              "--targets_per_batch", "16",
              "--accumulate_grad_batches", "1"]
basestr = "python ./training_scripts/train.py " + ' '.join(basestr)
# ffts = [True, False]
inits = ['isometric', 'tutte']
losses = ['l2']
datadirs = ['cylinder', 'cylinder_nocut']
distortions = ['dirichlet', 'edge']

f = open("slurm/cylinder_grad_exps", 'w')
for data in datadirs: 
    for distortion in distortions: 
        for init in inits:
            for loss in losses:
                expname = f"{data}_dist{distortion}_init{init}_loss{loss}"
                expstr = basestr + f" --init {init} --lossgradientstitching {loss} --root_dir_train ./data/{data} --root_dir_test ./data/{data} --expname {expname}"
                # if fft:
                #     expstr += " --fft --fft_dim 256"
                expstr += f" --lossdistortion {distortion}"
                f.write(expstr + "\n")
f.close() 

In [5]:
# Flat direct optimization experiments
basestr = "python directopt.py"

datadirs = ['cyl', 'cylnocut']
inits = ['isom']
# anneals = [True, False]
grads = [None, 'l2', 'split']

f = open("slurm/directopt_tests_v2", 'w')
for data in datadirs:
    for init in inits:
        for grad in grads:
            # Don't anneal if grad
            if anneal and grad is not None: 
                continue 
            expname = f"dopt_v2_data{data}_init{init}_grad{grad}"
            expstr = basestr + f" --vs ./scratch/data/{data}_vs.pt --fs ./scratch/data/{data}_fs.pt --init ./scratch/data/{data}_{init}.pt --savedir ./scratch/{expname}"
            if grad:
                expstr += f" --grad {grad}"
            f.write(expstr + "\n")
f.close() 


In [3]:
# Flat sanity check experiments
basestr = ["--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "2000",
             "--val_interval", "20", "--data_file", "cylinder.json", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--lr", "1e-5",
              "--layer_normalization", "FLATTEN",
              "--lossdistortion", "dirichlet", "--targets_per_batch", "16",
              "--accumulate_grad_batches", "1"]
basestr = "python ./training_scripts/train.py " + ' '.join(basestr)

datadirs = ['cylinder', 'cylinder_nocut']
inits = ['isometric', 'tutte']
sepdeltas = [0.1]
# schedules = [True, False]
grads = [None, 'l2', 'split']

f = open("slurm/cylinder_flat", 'w')
for data in datadirs:
    for delta in sepdeltas:
        for init in inits:
            for grad in grads:
                expname = f"flatcyl_sepd{delta}_data{data}_init{init}_grad{grad}"
                expstr = basestr + f" --root_dir_train ./data/{data} --root_dir_test ./data/{data} --init {init} --seplossdelta {delta} --expname {expname}"
                if grad:
                    expstr += f" --lossgradientstitching {grad}"
                else: 
                    expstr += f" --lossedgeseparation --seplossweight 1"
                f.write(expstr + "\n")
f.close() 


In [3]:
basestr = ["--root_dir_train", "./data/cylinder", "--root_dir_test", "./data/cylinder",
             "--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "8000",
             "--val_interval", "20", "--data_file", "cylinder.json", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--lr", "0.005",
              "--lossdistortion", "dirichlet", "--lossedgeseparation", "--seplossweight", "1", "--targets_per_batch", "16",
              "--accumulate_grad_batches", "1", "--fft", "--fft_dim", "256"]
basestr = "python ./training_scripts/train.py " + ' '.join(basestr)
sepdeltas = [0.01, 0.1, 0.5]
seplosses = ['l1', 'mse']

f = open("slurm/cylinder_exps", 'w')
for delta in sepdeltas:
    for seploss in seplosses:
        expname = f"cylinder_sepd{delta}_sepl{seploss}"
        expstr = basestr + f" --init isometric --eseploss {seploss} --seplossdelta {delta} --expname {expname}"
        f.write(expstr + "\n")
f.close() 

inits = ['isometric', 'tutte']
f = open("slurm/cylinder_exps_v2", 'w')
for init in inits:
    for seploss in seplosses:
        expname = f"cylinder_init{init}_sepl{seploss}"
        expstr = basestr + f" --init {init} --eseploss {seploss} --expname {expname} --seploss_schedule --seplossdelta_min 0.001"
        f.write(expstr + "\n")
f.close() 

# Same for bunny
basestr = ["--root_dir_train", "./data/bunny", "--root_dir_test", "./data/bunny",
             "--experiment_type", "DEFAULT", "--size_train", "1", "--size_test", "1", "--epochs", "8000",
             "--val_interval", "20", "--data_file", "bunny.json", "--align_2D", "--xp_type", "uv", "--gpu_strategy", "ddp",
              "--n_gpu", "1", "--no_poisson", "--identity", "--init", "isometric", "--lr", "0.005",
              "--lossdistortion", "dirichlet", "--lossedgeseparation", "--seplossweight", "1", "--targets_per_batch", "16",
              "--accumulate_grad_batches", "1"]
basestr = "python ./training_scripts/train.py " + ' '.join(basestr)
sepdeltas = [0.01, 0.1, 0.5]
seplosses = ['l1', 'mse']

f = open("slurm/bunny_exps", 'w')
for delta in sepdeltas:
    for seploss in seplosses:
        expname = f"bunny_sepd{delta}_sepl{seploss}"
        expstr = basestr + f" --eseploss {seploss} --seplossdelta {delta} --expname {expname}"
        f.write(expstr + "\n")
f.close() 

In [1]:
argstr = """--root_dir_train ./data/cylinder --root_dir_test ./data/cylinder --expname Debug --experiment_type DEFAULT 
        --size_train 1 --size_test 1 --epochs 5000 --val_interval 20 --data_file cylinder.json --align_2D --xp_type uv 
        --gpu_strategy ddp --n_gpu 1 --no_poisson --lossdistortion dirichlet --identity --init isometric --lossedgeseparation 
        --seplossdelta 0.1 --seplossweight 1 --targets_per_batch 16 --accumulate_grad_batches 1"""
print(" ".join(argstr.split()))

--root_dir_train ./data/cylinder --root_dir_test ./data/cylinder --expname Debug --experiment_type DEFAULT --size_train 1 --size_test 1 --epochs 5000 --val_interval 20 --data_file cylinder.json --align_2D --xp_type uv --gpu_strategy ddp --n_gpu 1 --no_poisson --lossdistortion dirichlet --identity --init isometric --lossedgeseparation --seplossdelta 0.5 --seplossweight 1 --targets_per_batch 16 --accumulate_grad_batches 1
